In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from scipy.ndimage.filters import gaussian_filter1d




global pattern groups

In [ ]:
# longterm decrease
pattern_0 = [('SB','low', 2), ('SB', 'medium', 4), ('SB', 'high', 3),('SB', 'super', 0), ('SD', 'low', 3), ('SD', 'medium', 0),
             ('SD','high', 1), ('SD', 'super', 0), ('MB', 'low', 1),('MB', 'medium', 2), ('MB', 'high', 1), ('MB', 'super', 0),
             ('MD','low', 2), ('MD', 'medium', 3), ('MD', 'high', 1),('MD', 'super', 0), ('MD', 'extreme', 0)]

# drought lonterm decrease + COVID increase
pattern_1 = [('SB','low', 1), ('SB', 'medium', 2), ('SB', 'high', 0),('SD', 'low', 0), ('SD', 'medium', 3)]

# longterm increase
pattern_2 = [('SB','low', 3), ('SB', 'medium', 3), ('SB', 'super', 3),('SD', 'low', 2), ('SD', 'medium', 1), ('SD', 'high', 0),
             ('MB','low', 0)]

# recession decrease + rebound
pattern_3 = [('SB','low', 0), ('SB', 'medium', 0), ('SB', 'high', 2), ('SD', 'low', 1), ('MB', 'low', 2)]

# drought rebound
pattern_4 = [('SB','high', 4), ('SB','high', 1), ('SB','super', 1), ('MB','medium', 1)]

# decrease recession + rebound after drought
pattern_5 = [('SB','medium', 1), ('MB','low', 3)]

# decrease after recession
pattern_6 = [('SB','super', 2), ('SD','high', 2), ('MD','medium', 1)]

COVID patterns

In [ ]:
covid_up = [('SB', 'low', 1), ('SB', 'medium', 2), ('SB', 'high', 4),('SB', 'high', 1),('SB', 'high', 0),('SB', 'super', 2),('SB', 'super', 3), ('SD', 'low', 0),
            ('SD', 'low', 3), ('SD', 'medium', 1), ('SD', 'medium', 3), ('SD','high', 2), ('SD', 'super', 0), ('SD', 'super', 1),
            ('MB', 'low', 0), ('MB', 'medium', 1), ('MD','low', 0), ('MD','low', 2), ('MD', 'medium', 2),  ('MD', 'extreme', 0)]


covid_down = [('SB', 'low', 0), ('SB', 'medium', 0),  ('SB', 'medium', 1), ('SB', 'high', 2),('SB', 'super', 1), 
              ('SD', 'low', 1), ('SD', 'low', 2), ('SD', 'medium', 2), ('SD', 'medium', 4), ('SD','high', 0), ('MB', 'low', 2), ('MB', 'low', 3), 
              ('MB', 'medium', 0),('MB', 'high', 0), ('MD','low', 1), ('MD','high', 0), ('MD', 'high', 1),  ('MD', 'super', 0)]

covid_no = [('SB', 'low', 2), ('SB', 'low', 3), ('SB', 'medium', 3),  ('SB', 'medium', 4), ('SB', 'high', 3),
            ('SB', 'super', 0),('SD', 'medium', 0), ('SD','high', 1), ('MB', 'low', 1),('MB', 'medium', 2),
            ('MB', 'high', 1), ('MB', 'super', 0), ('MD','medium', 0), ('MD','medium', 1), ('MD', 'medium', 3)]

# assign each customer their corresponding global pattern

In [ ]:
# load usage data

path_1 = '/content/drive/My Drive/Stanford-TUBerlin/Code Marie/MESA_2002_2020_cleaned.csv'

data = pd.read_csv(path_1, sep=',', index_col=0)

data['date']= pd.to_datetime(data['date'])

In [ ]:
cust = ['SB', 'SD', 'MB', 'MD']
values = [[0,1,2,3],[0,1,2,3],[0,1,2,3],[0,1,2,3,4]]
replacement = [['high', 'medium', 'super', 'low'], ['low', 'high', 'medium', 'super'],
               ['super', 'low', 'medium', 'high'], ['low', 'medium', 'extreme','high',  'super']]

all_profiles = pd.DataFrame(index= data['W_LocationID'].unique(), columns= ['group', 'consumption', 'pattern', 'Y'] )


In [ ]:
for i,_ in enumerate(cust):
  df = pd.read_pickle('/content/drive/MyDrive/Stanford-TUBerlin/CodePaper/Clustering/{}_clusters.pdf'.format(cust[i]))
  df.avg_consump.replace(values[i], replacement[i], inplace=True)
  consumption = df.avg_consump.values
  pattern = df.pattern.astype('int').values
  for k,l in enumerate(df.index):
    all_profiles.loc[l] = cust[i],consumption[k], pattern[k], np.NaN




In [ ]:
for idx, val in enumerate(pattern_0):
  x = all_profiles.loc[(all_profiles.group == val[0]) & (all_profiles.consumption == val[1]) & (all_profiles.pattern == val[2])]
  all_profiles.Y.loc[x.index] = 0

for idx, val in enumerate(pattern_1):
  x = all_profiles.loc[(all_profiles.group == val[0]) & (all_profiles.consumption == val[1]) & (all_profiles.pattern == val[2])]
  all_profiles.Y.loc[x.index] = 1

for idx, val in enumerate(pattern_2):
  x = all_profiles.loc[(all_profiles.group == val[0]) & (all_profiles.consumption == val[1]) & (all_profiles.pattern == val[2])]
  all_profiles.Y.loc[x.index] = 2

for idx, val in enumerate(pattern_3):
  x = all_profiles.loc[(all_profiles.group == val[0]) & (all_profiles.consumption == val[1]) & (all_profiles.pattern == val[2])]
  all_profiles.Y.loc[x.index] = 3

for idx, val in enumerate(pattern_4):
  x = all_profiles.loc[(all_profiles.group == val[0]) & (all_profiles.consumption == val[1]) & (all_profiles.pattern == val[2])]
  all_profiles.Y.loc[x.index] = 4

for idx, val in enumerate(pattern_5):
  x = all_profiles.loc[(all_profiles.group == val[0]) & (all_profiles.consumption == val[1]) & (all_profiles.pattern == val[2])]
  all_profiles.Y.loc[x.index] = 5

for idx, val in enumerate(pattern_6):
  x = all_profiles.loc[(all_profiles.group == val[0]) & (all_profiles.consumption == val[1]) & (all_profiles.pattern == val[2])]
  all_profiles.Y.loc[x.index] = 6

# ungrouped patterns will be group 7
all_profiles.Y.fillna(7, inplace=True)

In [ ]:
np.unique(all_profiles.Y, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([5263, 1445,  820,  592,  327,  308,   54,   62]))

In [ ]:
all_profiles

,group,consumption,pattern,Y
MW10002,MB,low,1,0
MW10003,MB,low,1,0
MW10006,MB,low,1,0
MW10007,MB,low,1,0
MW10008,MB,low,1,0
...,...,...,...,...
MW9991,MB,medium,0,7
MW9992,MB,medium,2,0
MW9993,MB,medium,2,0
MW9996,SB,medium,4,0


merge with features

In [ ]:
# get Census Tracts 
all_profiles['W_LocationID'] =all_profiles.index
data = data.drop_duplicates(subset='W_LocationID')

CT = data[['W_LocationID', 'CensusTract']].merge(all_profiles, on='W_LocationID', how='right')
CT_data = [f'{x:.2f}' for x in CT.CensusTract]
CT_data = [x.replace('.', '') for x in CT_data]
rest = ['0'] *len(CT_data)
CT.CensusTract = [i+str(j) for i,j in zip(rest, CT_data)]

In [ ]:
# get Block Group
CT_final = data[['W_LocationID', 'BlockGroup']].merge(CT, on='W_LocationID', how='right')
CT_data = [f'{x:.3f}' for x in CT_final.BlockGroup]
CT_data = [x.replace('.', '') for x in CT_data]
rest = ['060590'] *len(CT_data)
CT_final.BlockGroup = [i+str(j) for i,j in zip(rest, CT_data)]

In [ ]:
CT_final.rename(columns={'CensusTract': 'TRACTCE', 'BlockGroup': 'GEOID'}, inplace= True)
CT_final

,W_LocationID,GEOID,TRACTCE,group,consumption,pattern,Y
0,MW10002,060590636052,063605,MB,low,1,0
1,MW10003,060590636052,063605,MB,low,1,0
2,MW10006,060590636052,063605,MB,low,1,0
3,MW10007,060590636052,063605,MB,low,1,0
4,MW10008,060590636052,063605,MB,low,1,0
...,...,...,...,...,...,...,...
8866,MW9991,060590636051,063605,MB,medium,0,7
8867,MW9992,060590636051,063605,MB,medium,2,0
8868,MW9993,060590636051,063605,MB,medium,2,0
8869,MW9996,060590636051,063605,SB,medium,4,0


In [ ]:
# correct wrong BG numbers
wrong_BG =['060590639064', '060590638073', '060590638074', '060590639044', '060590632014', '060590636043']
new_BG = ['060590639063', '060590638072', '060590638072', '060590639043', '060590632013', '060590636042']

CT_final.GEOID.replace(wrong_BG, new_BG, inplace=True)

In [ ]:
block_feat = pd.read_pickle('/content/drive/MyDrive/Stanford-TUBerlin/Code Marie/ClusterChange_multiBaseline/GIS plots/Features_SocioDemo')

In [ ]:
block_feat.rename(columns={'Total':'totHH'}, inplace=True)

In [ ]:
merger = CT_final.merge(block_feat, on='GEOID', how='left')

In [ ]:
merger.replace(['-'], ['NaN'], inplace=True)

In [ ]:
merger.drop(columns=['GEOID', 'TRACTCE', 'W_LocationID'], inplace=True)

In [ ]:
merger.to_pickle('/content/drive/MyDrive/Stanford-TUBerlin/CodePaper/Mapping/FeatureMatrix2')